In [10]:
import sys
import random
import pickle

def get_player_move(player, board):
    try:
        move = input("Player " + str(player) + "'s turn: ")
        move = move.strip().split(',')
        move = [int(x) for x in move]
        return move
    except:
        print("error reading input")
        return False
        
    
def check_move_valid(player, move, board):
    if len(move) != 2:
        print("Cannot read move format!")
        return False
    
    for pos in move:
        if pos > 2:
            print("You moved outside the edges of the board!")
            return False
        
    board_pos = 3 * move[1] + move[0]
    if board[board_pos] != None:
        print("That spot was already taken!")
        return False
    
    return True

def add_move_to_board(player, move, board):
    board_pos = 3 * move[1] + move[0]
    board[board_pos] = player
    
# return value = [bool gameover, bool computerwin]
def check_game_over(board):
    winner = False
    game_over = False
    
    # check for x axis wins, so like x-x-x all in a row
    for x in range(0,3):
        if board[3*x] != None and board[3*x] == board[3*x+1] and board[3*x] == board[3*x+2]:
#             print("horizontal", x)
            game_over = True
            winner = board[3*x]
        
    # check for y axis wins, like above but vertically oriented
    for y in range(0,3):
        if board[y] != None and board[y] == board[y+3] and board[y] == board[y+6]:
#             print("vertical", y)
            game_over = True
            winner = board[y]
    
    # check for upwards diagonal wins, 0,2-1,1-2,0
    if board[4] != None and board[6] == board[4] and board[6] == board[2]:
#         print("up diag")
        game_over = True
        winner = board[4]
    
    # check for downwards diagonal wins, 0,0-1,1-2,2
    if board[4] != None and board[0] == board[4] and board[0] == board[8]:
#         print("dn diag")
        game_over = True
        winner = board[4]
        
    filled = True
    for x in range(0, len(board)):
        if board[x] == None:
            filled = False
            break
            
    if filled:
#         print("draw")
        game_over = True
        winner = None
            
    return [game_over, winner]


def switch_player(player):
    if player == 1:
        return 2
    else:
        return 1
    
# returns the move so we can add it to the list of moves for this game
def get_computer_move(player, board, past_boards):
#     board_pos = random_move(board)
    board_pos = move_from_past_moves(board, past_boards)
    
    move = [board_pos % 3, int(board_pos / 3)]
    add_move_to_board(player, move, board)
    return move

def move_from_past_moves(board, past_boards):
    current_board_key = past_boards_key(board)
    if current_board_key not in past_boards:
        return random_move(board)
    else:
        score_hash = {}
        
        # the array of point values corresponding to the value 
        # of each potential move when the board is in its current state
        # ex. [0, 0, -10, 0, 900, 0, 0, -10, 0]
        scores = past_boards[current_board_key]

        # create a hash table with the keys as the scores for this board state 
        # and the values are arrays of the indices into the board that have that score value
        for index in range(0, len(board)):
            score = scores[index]

            if score in score_hash:
                score_hash[score].append(index)
            else:
                score_hash[score] = [index]
                
            
        # shuffle each array of indices to prevent biasing towards upper left
        for score in score_hash.keys():
            random.shuffle(score_hash[score])
            
        
        # looking at the highest scores first, choose an available move from the list of board indices
        for score in sorted(list(score_hash.keys()), reverse=True):
            potential_moves = score_hash[score]
            for pos in potential_moves:
                if board[pos] == None:
                    return pos

                
def past_boards_key(board):
    boards_key = []
    for pos in board:
        if pos == None:
            boards_key.append('-')
        else:
            boards_key.append(str(pos))
    return ''.join(boards_key)
    
def random_move(board):
    available = []
    for x in range(0, len(board)):
        if board[x] == None:
            available.append(x)
            
    return random.choice(available)

def print_board(board):
    for y in range(0, 3):
        for x in range(0, 3):
            if board[3*y+x] == 1:
                sys.stdout.write(' X ')
            elif board[3*y+x] == 2:
                sys.stdout.write(' O ')
            else:
                sys.stdout.write('   ')
            
            if x != 2:
                sys.stdout.write('|')
        if y != 2:
            print("\n---+---+---")
        else:
            sys.stdout.write("\n")
    

def score_past_boards(past_boards, this_games_input, game_over_result):
    # take each move in the game input list
    # create a board from the cumulative moves
    # when the current move is a computer move
        # get the key for this board
        # if there is a board in past boards for this key,
            # score the move position based on what the outcome of the game was  
    this_game_board = [None] * 9
    for game_input in this_games_input:
        board_pos = 3*game_input['move'][1] + game_input['move'][0]

        current_board_key = past_boards_key(this_game_board)
        if current_board_key not in past_boards:
            past_boards[current_board_key] = [0] * 9

        if game_over_result[1] == game_input['player']:
            past_boards[current_board_key][board_pos] += 100 # win
        elif game_over_result[1] != None:
            past_boards[current_board_key][board_pos] -= 10 # loss
        else:
            past_boards[current_board_key][board_pos] -= 1 # draw
            
        this_game_board[board_pos] = game_input['player']
        
    
    return past_boards
            

def game_loop(past_boards, computer_only, analytics):
    board = [None] * 9
    player = 1
    this_games_moves = []
    game_over_result = [False]

    while not game_over_result[0]:
        current_board_key = past_boards_key(board)
        if current_board_key in past_boards:
            print("Recommended play:", past_boards[current_board_key])
            
        if computer_only:
            this_games_moves.append({'player':player, 'move': get_computer_move(player, board, past_boards)})
        else:
            if player == 2:
                print("Computer's turn...")
                this_games_moves.append({'player':player, 'move': get_computer_move(player, board, past_boards)})
            else:
                player_move = get_player_move(player, board)
                if player_move != False and check_move_valid(player, player_move, board):  
                    this_games_moves.append({'player':player, 'move':player_move})
                    add_move_to_board(player, player_move, board)
                else:
                    break
            
        player = switch_player(player)
        if not computer_only:
            print_board(board)
            
        game_over_result = check_game_over(board)
        
    if game_over_result[0]:
        if game_over_result[1] == 2:
            analytics['player 2 wins'] += 1
            if not computer_only:
                print("Computer wins!")
        elif game_over_result[1] == 1:
            analytics['player 1 wins'] += 1
            if not computer_only:
                print("Player 1 wins!")
        else:
            analytics['draws'] += 1
            if not computer_only:
                print("Cat's game! (Draw)")
            
        
    if game_over_result[1] != None:
        past_boards = score_past_boards(past_boards, this_games_moves, game_over_result)
        
    if not computer_only:
        continue_input = input("Game over! Play again? [Y/n] ") 
        if continue_input == '' or continue_input == 'Y' or continue_input == 'y':
            return True
        return False
   

if __name__ == "__main__":
    computer_only = False
    game_count = 100000
    analytics = {
        'player 1 wins': 0,
        'player 2 wins': 0,
        'draws': 0
    }
    
    try:
        with open('saved_moves.txt', 'rb') as handle:
            past_boards = pickle.loads(handle.read())
#             past_boards = {}
            
    except FileNotFoundError:
        past_boards = {}
        
        
    if computer_only:
        for x in range(0, game_count):
            game_loop(past_boards, computer_only, analytics)
            if x % 5000 == 0:
                print("Game #", x, "...")
    else:
        print("Enter your moves in the format 0,0 to place your mark on the upper left corner of the board")
        while game_loop(past_boards, computer_only, analytics):
            pass
    
    with open('saved_moves.txt', 'wb') as handle:
        pickle.dump(past_boards, handle)
    
    print(analytics)
    
    
    

Enter your moves in the format 0,0 to place your mark on the upper left corner of the board
Recommended play: [11000, -10, 0, 0, 200, 0, 0, 0, 0]
Player 1's turn: 0,0
 X |   |   
---+---+---
   |   |   
---+---+---
   |   |   
Recommended play: [0, -10, -10, -10, 1930, -10, 0, -10, -10]
Computer's turn...
 X |   |   
---+---+---
   | O |   
---+---+---
   |   |   
Recommended play: [0, 0, -10, 4480, 0, 0, 0, 0, -10]
Player 1's turn: 0,1
 X |   |   
---+---+---
 X | O |   
---+---+---
   |   |   
Recommended play: [0, -10, 0, 0, 0, 0, 1760, -10, -10]
Computer's turn...
 X |   |   
---+---+---
 X | O |   
---+---+---
 O |   |   
Recommended play: [0, -20, 2260, 0, 0, -20, 0, -10, -10]
Player 1's turn: 2,0
 X |   | X 
---+---+---
 X | O |   
---+---+---
 O |   |   
Recommended play: [0, 200, 0, 0, 0, -10, 0, -10, -10]
Computer's turn...
 X | O | X 
---+---+---
 X | O |   
---+---+---
 O |   |   
Recommended play: [0, 0, 0, 0, 0, -10, 0, 0, -10]
Player 1's turn: 2,2
 X | O | X 
---+---+---